# Tonne-Kilometer (TKM) Calculator

This notebook analyzes route data from the TSS optimization process to calculate and verify transport-kilometer metrics. It processes both direct routes (Project → CCH) and indirect routes via temporary storage sites (Project → TSS → CCH).

## Key Metrics
- **Total Distance**: Sum of all route segments in kilometers
- **Tonne-kilometers (TKM)**: Distance × tonnage for each route
- **Route Types**:
  - Direct to CCH: Single leg journey (Project → CCH)
  - Via TSS: Two-leg journey (Project → TSS → CCH)


In [ ]:
# 1. Import Libraries
import pandas as pd
import os

## Data Loading and Preparation
In this section, we:
1. Import required libraries
2. Set up file paths
3. Load and validate the input data
4. Clean the data by handling missing values and invalid entries

In [ ]:
# 2. Define Input File Path (use absolute path)
input_file_path = 'output/final_origin_routes.csv'

In [ ]:
# 3. Read Input CSV
try:
    df = pd.read_csv(input_file_path)
    print(f"Successfully read {len(df)} rows from {input_file_path}")
except FileNotFoundError:
    print(f"Error: Input file not found at {input_file_path}")
    df = None
except Exception as e:
    print(f"Error reading CSV file: {e}")
    df = None

In [ ]:
# 4. Data Cleaning
if df is not None:
    original_rows = len(df)
    # Convert relevant columns to numeric, coercing errors
    df['tonnage'] = pd.to_numeric(df['tonnage'], errors='coerce')
    df['total_tonne_km'] = pd.to_numeric(df['total_tonne_km'], errors='coerce')
    df['total_dist'] = pd.to_numeric(df['total_dist'], errors='coerce')
    df['leg1_dist'] = pd.to_numeric(df['leg1_dist'], errors='coerce')
    df['leg2_dist'] = pd.to_numeric(df['leg2_dist'], errors='coerce')

    # Drop rows with NaN in critical columns after coercion
    # For all rows, check basic requirements
    df.dropna(subset=['tonnage', 'total_tonne_km', 'total_dist', 'status', 'leg1_dist'], inplace=True)
    
    # For Via_TSS routes, also check leg2_dist
    df.loc[df['status'] == 'Via_TSS'].dropna(subset=['leg2_dist'], inplace=True)

    # Filter out zero or negative tonnage
    df = df[df['tonnage'] > 0].copy() # Use .copy() to avoid SettingWithCopyWarning

    cleaned_rows = len(df)
    if original_rows > cleaned_rows:
        print(f"Cleaned {original_rows - cleaned_rows} rows with invalid/zero data.")

    print(f"Data cleaning complete. {cleaned_rows} rows remaining.")
else:
    print("DataFrame is None, skipping data cleaning.")

In [ ]:
# 5. Project Counts
if df is not None and not df.empty:
    projects_direct_count = len(df[df['status'] == 'Direct_CCH'])
    projects_tss_count = len(df[df['status'] == 'Via_TSS'])
    total_projects = projects_direct_count + projects_tss_count

    print("\n--- Project Summary ---")
    print(f"Total Projects Analyzed: {total_projects}")
    print(f"Direct to CCH: {projects_direct_count}")
    print(f"Via TSS: {projects_tss_count}")
else:
    print("DataFrame is None or empty, skipping project counts.")

## Analysis Overview
We'll analyze the routing data in several steps:
1. Count projects by route type (Direct vs. TSS)
2. Calculate detailed statistics for each route type
3. Verify distance and TKM calculations
4. Generate summary tables for CCH and TSS routes

In [ ]:
# 6. Data Analysis and Statistics
if df is not None and not df.empty:
    # Separate data by status
    df_direct = df[df['status'] == 'Direct_CCH'].copy()
    df_tss = df[df['status'] == 'Via_TSS'].copy()

    print("\n--- Table 1: Statistics for CCH (Direct Deliveries) ---")
    if not df_direct.empty:
        cch_stats = df_direct.groupby('leg1_loc').agg(
            Total_Distance_km=('total_dist', 'sum'),
            Mean_Distance_km=('total_dist', 'mean'),
            Projects_Served=('origin', 'count'), # Assuming 'origin' uniquely identifies a project route
            Tonnage_t=('tonnage', 'sum'),
            Total_tkm=('total_tonne_km', 'sum')
        ).reset_index()

        cch_stats.rename(columns={
            'leg1_loc': 'Location',
            'Total_Distance_km': 'Total Distance (km)',
            'Mean_Distance_km': 'Mean Distance (km)',
            'Projects_Served': 'Projects Served',
            'Tonnage_t': 'Tonnage (t)',
            'Total_tkm': 'Total t-km'
        }, inplace=True)

        # Add Totals row
        if not cch_stats.empty:
            totals_cch = {
                'Location': 'Totals',
                'Total Distance (km)': cch_stats['Total Distance (km)'].sum(),
                'Mean Distance (km)': df_direct['total_dist'].mean() if not df_direct.empty else 0,
                'Projects Served': cch_stats['Projects Served'].sum(),
                'Tonnage (t)': cch_stats['Tonnage (t)'].sum(),
                'Total t-km': cch_stats['Total t-km'].sum()
            }
            df_totals = pd.DataFrame([totals_cch])
            cch_stats = pd.concat([cch_stats, df_totals], ignore_index=True)

            print(cch_stats.to_string(index=False, float_format='{:,.2f}'.format, na_rep='-'))
        else:
            print("No data to aggregate for direct CCH deliveries.")
    else:
        print("No data available for direct CCH deliveries.")

    print("\n--- Table 2: Statistics for TSS Routes ---")
    if not df_tss.empty:
        # Calculate Leg 1 and Leg 2 t-km for TSS routes
        df_tss['leg1_tkm'] = df_tss['tonnage'] * df_tss['leg1_dist']
        df_tss['leg2_tkm'] = df_tss['tonnage'] * df_tss['leg2_dist']

        tss_stats = df_tss.groupby('leg1_loc').agg(
            Tonnage_t=('tonnage', 'sum'),
            Mean_Total_Distance_km=('total_dist', 'mean'),
            Projects_Served=('origin', 'count'), # Assuming 'origin' uniquely identifies a project route
            Mean_Leg1_Distance_km=('leg1_dist', 'mean'),
            Mean_Leg2_Distance_km=('leg2_dist', 'mean'),
            Total_Leg1_tkm=('leg1_tkm', 'sum'),
            Total_Leg2_tkm=('leg2_tkm', 'sum'),
            Total_tkm=('total_tonne_km', 'sum')
        ).reset_index()

        tss_stats.rename(columns={
            'leg1_loc': 'TSS Name',
            'Tonnage_t': 'Tonnage (t)',
            'Mean_Total_Distance_km': 'Mean Total Distance (km)',
            'Projects_Served': 'Projects Served',
            'Mean_Leg1_Distance_km': 'Mean d_{P->TSS,i} (km)',
            'Mean_Leg2_Distance_km': 'Mean d_{TSS->CCH,i} (km)',
            'Total_Leg1_tkm': 'Leg 1 (t-km)',
            'Total_Leg2_tkm': 'Leg 2 (t-km)',
            'Total_tkm': 'Total t-km'
        }, inplace=True)

        # Define the desired column order for the TSS table
        tss_column_order = [
            'TSS Name',
            'Tonnage (t)',
            'Mean Total Distance (km)',
            'Projects Served',
            'Mean d_{P->TSS,i} (km)',
            'Mean d_{TSS->CCH,i} (km)',
            'Leg 1 (t-km)',
            'Leg 2 (t-km)',
            'Total t-km'
        ]
        tss_stats = tss_stats[tss_column_order]

        # Add Totals row
        if not tss_stats.empty:
            totals_tss = {
                'TSS Name': 'Totals',
                'Tonnage (t)': tss_stats['Tonnage (t)'].sum(),
                'Mean Total Distance (km)': df_tss['total_dist'].mean() if not df_tss.empty else 0,
                'Projects Served': tss_stats['Projects Served'].sum(),
                'Mean d_{P->TSS,i} (km)': df_tss['leg1_dist'].mean() if not df_tss.empty else 0,
                'Mean d_{TSS->CCH,i} (km)': df_tss['leg2_dist'].mean() if not df_tss.empty else 0,
                'Leg 1 (t-km)': tss_stats['Leg 1 (t-km)'].sum(),
                'Leg 2 (t-km)': tss_stats['Leg 2 (t-km)'].sum(),
                'Total t-km': tss_stats['Total t-km'].sum()
            }
            df_totals = pd.DataFrame([totals_tss])
            tss_stats = pd.concat([tss_stats, df_totals], ignore_index=True)

            print(tss_stats.to_string(index=False, float_format='{:,.2f}'.format, na_rep='-'))
        else:
            print("No data to aggregate for TSS routes.")
    else:
        print("No data available for TSS routes.")

else:
    print("DataFrame is None or empty, skipping analysis.")

## Detailed Statistics
This section generates two main tables:
1. **CCH Direct Routes**: Statistics for direct deliveries to each CCH
2. **TSS Routes**: Detailed metrics for routes going through each TSS, including:
   - Tonnage handled
   - Average distances for each leg
   - TKM calculations by leg
   - Total combined metrics

In [ ]:
# 7. Manual Tonne-Kilometer Calculations and Analysis
if df is not None and not df.empty:
    # Create copy of dataframe for calculations
    df_calc = df.copy()
    
    # Use distances directly as they are already in kilometers
    df_calc['total_dist_km'] = df_calc['total_dist']
    df_calc['leg1_dist_km'] = df_calc['leg1_dist']
    df_calc['leg2_dist_km'] = df_calc['leg2_dist']
    
    # Calculate t-km for all routes
    df_calc['total_tonne_km'] = df_calc['tonnage'] * df_calc['total_dist_km']
    
    # For TSS routes, calculate leg-specific t-km
    df_calc['leg1_tkm'] = df_calc['tonnage'] * df_calc['leg1_dist_km']
    df_calc['leg2_tkm'] = df_calc.apply(
        lambda row: row['tonnage'] * row['leg2_dist_km'] if row['status'] == 'Via_TSS' else 0, 
        axis=1
    )
    
    print("\n--- Manual Tonne-Kilometer Analysis ---")
    
    # Analyze Direct CCH routes
    print("\n--- Table 1: Statistics for CCH (Direct Deliveries) ---")
    df_direct = df_calc[df_calc['status'] == 'Direct_CCH'].copy()
    if not df_direct.empty:
        cch_stats = df_direct.groupby('leg1_loc').agg(
            Total_Distance_km=('total_dist_km', 'sum'),
            Mean_Distance_km=('total_dist_km', 'mean'),
            Projects_Served=('origin', 'count'),
            Tonnage_t=('tonnage', 'sum'),
            Total_tkm=('total_tonne_km', 'sum')
        ).reset_index()

        cch_stats.rename(columns={
            'leg1_loc': 'Location',
            'Total_Distance_km': 'Total Distance (km)',
            'Mean_Distance_km': 'Mean Distance (km)',
            'Projects_Served': 'Projects Served',
            'Tonnage_t': 'Tonnage (t)',
            'Total_tkm': 'Total t-km'
        }, inplace=True)

        # Add Totals row
        if not cch_stats.empty:
            totals_cch = {
                'Location': 'Totals',
                'Total Distance (km)': cch_stats['Total Distance (km)'].sum(),
                'Mean Distance (km)': df_direct['total_dist_km'].mean(),
                'Projects Served': cch_stats['Projects Served'].sum(),
                'Tonnage (t)': cch_stats['Tonnage (t)'].sum(),
                'Total t-km': cch_stats['Total t-km'].sum()
            }
            df_totals = pd.DataFrame([totals_cch])
            cch_stats = pd.concat([cch_stats, df_totals], ignore_index=True)
            print(cch_stats.to_string(index=False, float_format='{:,.2f}'.format, na_rep='-'))
        else:
            print("No data to aggregate for direct CCH deliveries.")
    else:
        print("No data available for direct CCH deliveries.")

    # Analyze TSS routes
    print("\n--- Table 2: Statistics for TSS Routes ---")
    df_tss = df_calc[df_calc['status'] == 'Via_TSS'].copy()
    if not df_tss.empty:
        # Extract TSS number for sorting and ensure it's a copy
        df_tss = df_tss.assign(tss_num=df_tss['leg1_loc'].str.extract(r'(\d+)').astype(float))
        
        tss_stats = df_tss.groupby('leg1_loc').agg(
            Tonnage_t=('tonnage', 'sum'),
            Mean_Total_Distance_km=('total_dist_km', 'mean'),
            Projects_Served=('origin', 'count'),
            Mean_Leg1_Distance_km=('leg1_dist_km', 'mean'),
            Mean_Leg2_Distance_km=('leg2_dist_km', 'mean'),
            Total_Leg1_tkm=('leg1_tkm', 'sum'),
            Total_Leg2_tkm=('leg2_tkm', 'sum'),
            Total_tkm=('total_tonne_km', 'sum'),
            tss_num=('tss_num', 'first')  # Keep the TSS number for sorting
        ).reset_index()
        
        # Sort by TSS number
        tss_stats = tss_stats.sort_values('tss_num').drop('tss_num', axis=1)

        tss_stats.rename(columns={
            'leg1_loc': 'TSS Name',
            'Tonnage_t': 'Tonnage (t)',
            'Mean_Total_Distance_km': 'Mean Total Distance (km)',
            'Projects_Served': 'Projects Served',
            'Mean_Leg1_Distance_km': 'Mean d_{P->TSS,i} (km)',
            'Mean_Leg2_Distance_km': 'Mean d_{TSS->CCH,i} (km)',
            'Total_Leg1_tkm': 'Leg 1 (t-km)',
            'Total_Leg2_tkm': 'Leg 2 (t-km)',
            'Total_tkm': 'Total t-km'
        }, inplace=True)

        # Define column order
        tss_column_order = [
            'TSS Name',
            'Tonnage (t)',
            'Mean Total Distance (km)',
            'Projects Served',
            'Mean d_{P->TSS,i} (km)',
            'Mean d_{TSS->CCH,i} (km)',
            'Leg 1 (t-km)',
            'Leg 2 (t-km)',
            'Total t-km'
        ]
        tss_stats = tss_stats[tss_column_order]

        # Add Totals row
        if not tss_stats.empty:
            totals_tss = {
                'TSS Name': 'Totals',
                'Tonnage (t)': tss_stats['Tonnage (t)'].sum(),
                'Mean Total Distance (km)': df_tss['total_dist_km'].mean(),
                'Projects Served': tss_stats['Projects Served'].sum(),
                'Mean d_{P->TSS,i} (km)': df_tss['leg1_dist_km'].mean(),
                'Mean d_{TSS->CCH,i} (km)': df_tss['leg2_dist_km'].mean(),
                'Leg 1 (t-km)': tss_stats['Leg 1 (t-km)'].sum(),
                'Leg 2 (t-km)': tss_stats['Leg 2 (t-km)'].sum(),
                'Total t-km': tss_stats['Total t-km'].sum()
            }
            df_totals = pd.DataFrame([totals_tss])
            tss_stats = pd.concat([tss_stats, df_totals], ignore_index=True)
            print(tss_stats.to_string(index=False, float_format='{:,.2f}'.format, na_rep='-'))

            # Verify leg1 + leg2 equals total for each TSS
            leg_sum = tss_stats['Leg 1 (t-km)'] + tss_stats['Leg 2 (t-km)']
            if not (abs(leg_sum - tss_stats['Total t-km']) <= 0.01).all():
                print("\nWarning: Sum of legs differs from total t-km for some TSS locations")
        else:
            print("No data to aggregate for TSS routes.")
    else:
        print("No data available for TSS routes.")
    
    # Print grand total
    if not df_calc.empty:
        print("\nGrand Total Summary:")
        print(f"Total Projects: {len(df_calc)}")
        print(f"Total Distance: {df_calc['total_dist_km'].sum():,.2f} km")
        print(f"Total Tonnage: {df_calc['tonnage'].sum():,.2f} t")
        print(f"Grand Total t-km: {df_calc['total_tonne_km'].sum():,.2f} t-km")

else:
    print("DataFrame is None or empty, skipping t-km calculations and analysis.")

## Manual TKM Verification
This section performs independent calculations to verify the TKM values:
- Recalculates distances and TKM for each route type
- Compares calculated values with provided data
- Breaks down metrics by route segment
- Generates summary statistics for validation

In [ ]:
# 8. Distance Calculation Verification
if df is not None and not df.empty:
    # Create copy of dataframe for verification
    df_verify = df.copy()
    
    # Use distances directly as they are already in kilometers
    df_verify['leg1_dist_km'] = df_verify['leg1_dist']
    df_verify['leg2_dist_km'] = df_verify['leg2_dist']
    df_verify['total_dist_km'] = df_verify['total_dist']
    
    # Calculate total distance manually for verification
    df_verify['calculated_total_dist'] = df_verify.apply(
        lambda row: row['leg1_dist_km'] if row['status'] == 'Direct_CCH' 
                   else (row['leg1_dist_km'] + row['leg2_dist_km']), 
        axis=1
    )
    
    print("\n=== Distance Calculation Verification ===")
    print("Note: All distances in kilometers")
    
    # Verify Direct CCH routes
    print("\nDirect CCH Routes:")
    direct_mask = df_verify['status'] == 'Direct_CCH'
    direct_df = df_verify[direct_mask].copy()
    print("Example route distances:")
    print(direct_df[['origin', 'leg1_loc', 'leg1_dist_km', 'total_dist_km']].head())
    print(f"\nSummary:")
    print(f"Number of routes: {len(direct_df)}")
    print(f"Average leg1 distance (Project->CCH): {direct_df['leg1_dist_km'].mean():.2f} km")
    print(f"Total distance: {direct_df['total_dist_km'].sum():.2f} km")
    print(f"Total calculated distance: {direct_df['calculated_total_dist'].sum():.2f} km")
    
    # Verify TSS routes
    print("\nVia TSS Routes:")
    tss_mask = df_verify['status'] == 'Via_TSS'
    tss_df = df_verify[tss_mask].copy()
    print("Example route distances:")
    print(tss_df[['origin', 'leg1_loc', 'leg1_dist_km', 'leg2_dist_km', 'total_dist_km']].head())
    print(f"\nSummary:")
    print(f"Number of routes: {len(tss_df)}")
    print(f"Average leg1 distance (Project->TSS): {tss_df['leg1_dist_km'].mean():.2f} km")
    print(f"Average leg2 distance (TSS->CCH): {tss_df['leg2_dist_km'].mean():.2f} km")
    print(f"Average total distance: {tss_df['total_dist_km'].mean():.2f} km")
    print(f"Average calculated total: {tss_df['calculated_total_dist'].mean():.2f} km")
    
    # Calculate and print total distances for each TSS
    print("\nTSS-wise Distance Summary:")
    # Extract TSS number for sorting and ensure it's a copy
    tss_df = tss_df.assign(tss_num=tss_df['leg1_loc'].str.extract(r'(\d+)').astype(float))
    
    tss_distance_summary = tss_df.groupby('leg1_loc').agg({
        'leg1_dist_km': 'sum',
        'leg2_dist_km': 'sum',
        'origin': 'count',
        'tss_num': 'first'  # Keep the TSS number for sorting
    }).reset_index()
    
    # Sort by TSS number
    tss_distance_summary = tss_distance_summary.sort_values('tss_num')
    tss_distance_summary.drop('tss_num', axis=1, inplace=True)
    
    tss_distance_summary.columns = ['TSS Name', 'Total Leg1 Distance (km)', 'Total Leg2 Distance (km)', 'Number of Routes']
    tss_distance_summary['Total Combined Distance (km)'] = tss_distance_summary['Total Leg1 Distance (km)'] + tss_distance_summary['Total Leg2 Distance (km)']
    
    # Add totals row
    totals = pd.DataFrame([{
        'TSS Name': 'TOTAL',
        'Total Leg1 Distance (km)': tss_distance_summary['Total Leg1 Distance (km)'].sum(),
        'Total Leg2 Distance (km)': tss_distance_summary['Total Leg2 Distance (km)'].sum(),
        'Number of Routes': tss_distance_summary['Number of Routes'].sum(),
        'Total Combined Distance (km)': tss_distance_summary['Total Combined Distance (km)'].sum()
    }])
    tss_distance_summary = pd.concat([tss_distance_summary, totals], ignore_index=True)
    print(tss_distance_summary.to_string(index=False, float_format='{:,.2f}'.format))
    
    # Verify distance calculation accuracy
    print("\nDistance Calculation Verification:")
    df_verify['dist_difference'] = abs(df_verify['total_dist_km'] - df_verify['calculated_total_dist'])
    has_difference = df_verify['dist_difference'] > 0.01  # Check differences > 0.01 km
    if has_difference.any():
        print("Found differences between provided and calculated distances:")
        print(df_verify[has_difference][['origin', 'status', 'total_dist_km', 'calculated_total_dist', 'dist_difference']])
    else:
        print("✓ All calculated distances match provided distances (within 0.01 km)")
    
    # Print overall statistics
    print("\nOverall Distance Statistics:")
    print(f"Total distance (all routes): {df_verify['total_dist_km'].sum():.2f} km")
    print(f"Direct routes total: {direct_df['total_dist_km'].sum():.2f} km")
    print(f"TSS routes total: {tss_df['total_dist_km'].sum():.2f} km")
    
    # Print distance formulas used
    print("\nDistance Calculation Formulas:")
    print("1. Direct CCH routes:")
    print("   total_distance = leg1_distance")
    print("   (Project -> CCH)")
    print("\n2. Via TSS routes:")
    print("   total_distance = leg1_distance + leg2_distance")
    print("   (Project -> TSS -> CCH)")
    print("\nNote: All distances are in kilometers")

else:
    print("DataFrame is None or empty, skipping distance verification.")

## Distance Calculation Validation
Final verification of all distance calculations:
- Validates route distances for both Direct and TSS routes
- Compares segment distances with total distances
- Generates TSS-wise distance summaries
- Identifies any discrepancies in calculations